https://github.com/prem2017/new-entity-labelling/blob/master/new_entity_labelling.ipynb

# Init Environment

In [1]:
# !pip install en_core_web_sm
# !python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_sm
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
from src.config import *
import pandas as pd
import re

import spacy
from spacy import displacy
from spacy.tokens import Doc, Span
from spacy.language import Language
from src.helper_domain import * 

In [10]:
if 'df_consolidated' not in locals():
    data_url = f'{DATA_FOLDER_PATH_PROCESSED}/data_consolidated.xlsx'
    df_consolidated = pd.read_excel(data_url)
    
df_samples = df_consolidated.sample(100, random_state=42)

# filter the samples with the Title containing the word 'data'
df_samples = df_consolidated[df_consolidated['Title_Translated'].str.contains('ODRS')]
df_samples = df_samples.sample(min(20, df_samples.shape[0]), random_state=42)

# Domain Specific Named Entity Recognition (NER)

In [7]:
# Load the pre-trained spaCy model
nlp = spacy.load("en_core_web_lg")
# Set up the pipeline for training
ner = nlp.get_pipe("ner")

# Add the custom entity recognizer to the pipeline
nlp.add_pipe("custom_entity_setter", last=True)

<function src.helper_domain.set_custom_entities(doc)>

In [11]:
# Example rendering entities by color only without labels
# colors = {
#     'ORG': 'linear-gradient(90deg, #aa9cfc, #fc9ce7)', 
#     'GPE': 'radial-gradient(yellow, green)',
#     'PRODUCT': 'radial-gradient(yellow, green)',
#     }
# options = {'ents': ['ORG', 'GPE', 'PRODUCT'], 'colors': colors}


# test the trained model before training
for text in df_samples['Title_Enhanced']:
    # print(text)
    doc = nlp(text)
    if doc.ents:
        displacy.render(doc, style='ent')
    else:
        displacy.render(doc)

trouble shotting

In [37]:
text = 'Cannot access Delfi Portal due to phone number...'
text = 'Completed PETRONAS DELFI How to display core log images in WSW'
text = 'Agora Agora gateway to mexico'
doc = nlp(text)
if doc.ents is not None:
    displacy.render(doc, style='ent')